In [1]:
# 01_limpieza_datos.ipynb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



In [2]:
# 1. Cargar datos simulados
df = pd.read_csv('../data/datos_credito_simulados.csv')

print("Datos originales:", df.shape)



Datos originales: (1000, 16)


In [3]:
# 2. Mapeo de Variables Ordinales (Donde el orden importa)
# Según Apéndice A: AAA es mejor que AA, etc. [cite: 342]
mapa_score = {
    'AAA': 5, 'AA': 4, 'A': 3, 
    'Analista': 2, 'Rechazado': 1
}
df['score_interno_num'] = df['score_interno'].map(mapa_score)



In [4]:
# 3. Tratamiento de Variables Categóricas Nominales (One-Hot Encoding)
# Convertimos Segmento, Producto, Garantia, Estado Civil a columnas binarias
cols_categoricas = ['segmento_credito', 'producto', 'garantia', 'estado_civil', 'destino_credito']
# Nota: Si alguna columna no existe en el simulado, pandas la ignora o ajustamos
df = pd.get_dummies(df, columns=['segmento_credito', 'producto', 'garantia', 'estado_civil'], drop_first=True)



In [5]:
# 4. Selección de Variables Finales (Features X y Target y)
# Eliminamos columnas de texto originales que ya transformamos
X = df.drop(['riesgo_real', 'score_interno'], axis=1) 
y = df['riesgo_real']



In [6]:
# 5. División de Datos (70% Entrenamiento, 30% Prueba)
# Esto es estándar para validar el modelo [cite: 417]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)



In [7]:
# 6. Escalado de Datos (Normalización)
# Importante para algoritmos como Regresión Logística y Redes Neuronales [cite: 447]
scaler = StandardScaler()
vars_numericas = ['dias_mora_prom', 'edad', 'ingreso_mensual', 'ventas_anuales', 'monto_solicitado', 'plazo_meses']

# Solo ajustamos el escalador con datos de TRAIN para evitar "data leakage"
X_train[vars_numericas] = scaler.fit_transform(X_train[vars_numericas])
X_test[vars_numericas] = scaler.transform(X_test[vars_numericas])



In [9]:
# 7. Guardar datasets procesados para el siguiente paso
X_train.to_csv('../data/X_train.csv', index=False)
X_test.to_csv('../data/X_test.csv', index=False)
y_train.to_csv('../data/y_train.csv', index=False)
y_test.to_csv('../data/y_test.csv', index=False)

# Guardar el escalador para usarlo luego en la App Web
import joblib
joblib.dump(scaler, '../web_app/credit_risk/ml_models/scaler.pkl')

print("¡Limpieza completada! Archivos guardados en /data y scaler en /web_app")

¡Limpieza completada! Archivos guardados en /data y scaler en /web_app
